In [3]:
import os
import sys
import h5py
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
from modis_utils.misc import cache_data, restore_data

In [5]:
data = restore_data(os.path.join('cache', 'boundary_vectors_ALL_1.dat'))
data_train = data[0]
data_test = data[1]
data_train.shape, data_test.shape

((598, 1484, 2), (92, 1484, 2))

In [6]:
def mse(x, y):
    if isinstance(x, pd.Series) or isinstance(x, pd.DataFrame):
        x = x.values
    if isinstance(y, pd.Series) or isinstance(y, pd.DataFrame):
        y = y.values
    return np.mean((x - y)**2)

def mse_1(x, y):
    a = (x - y)**2
    return np.mean(a, axis=0)

def mse_2(x, y):
    a = (x - y)**2
    b = np.where(a > 30, 0, 1)
    c = a*b
    return np.sum(c)/np.sum(b)

def save_data_pickle(data, path):
    with open(path, 'wb') as f:
        pickle.dump(data, f)

def load_data_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

In [7]:
convlstm_inference = restore_data('../simple_63/convlstm_predict_boundary_vectors.dat')

In [8]:
convlstm_inference.shape

(138, 1657, 2)

In [9]:
convlstm_predict = convlstm_inference[-92:]
convlstm_predict.shape

(92, 1657, 2)

In [10]:
pos = [  41,   42,   47,   48,  105,  109,  120,  121,  122,  123,  124,
         140,  141,  142,  146,  147,  153,  154,  167,  168,  172,  173,
         179,  180,  181,  182,  191,  192,  193,  194,  198,  205,  206,
         207,  219,  223,  228,  229,  230,  231,  243,  244,  248,  249,
         252,  253,  254,  265,  266,  270,  271,  275,  276,  287,  306,
         316,  317,  318,  319,  325,  326,  330,  334,  335,  336,  337,
         338,  352,  353,  396,  415,  424,  425,  480,  481,  482,  483,
         484,  485,  486,  505,  506,  507,  508,  523,  525,  526,  527,
         528,  529,  530,  531,  532,  533,  534,  535,  550,  566,  581,
         607,  608,  609,  610,  611,  837,  843,  844,  845,  846,  847,
         848,  849,  850,  851,  852,  855,  856,  905,  906,  907,  933,
        1184, 1186, 1188, 1267, 1268, 1272, 1273, 1274, 1276, 1277, 1278,
        1279, 1473, 1475, 1476, 1477, 1479, 1480, 1481, 1482, 1483, 1484,
        1485, 1486, 1487, 1488, 1489, 1490, 1491, 1492, 1493, 1494, 1495,
        1496, 1497, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1505, 1506,
        1507, 1508, 1510, 1511, 1515, 1519, 1523, 1535]

In [11]:
a = np.zeros(convlstm_predict.shape[1])
for x in pos:
    a[x] = 1

In [12]:
convlstm_predict_1 = []
for i in range(convlstm_predict.shape[1]):
    if a[i] == 0:
        convlstm_predict_1.append(convlstm_predict[:, i, :])
convlstm_predict_1 = np.asarray(convlstm_predict_1)
convlstm_predict_1.shape

(1484, 92, 2)

In [13]:
convlstm_predict_1 = convlstm_predict_1.transpose((1,0,2))
convlstm_predict_1.shape

(92, 1484, 2)

In [14]:
mse(convlstm_predict_1, data_test)

15.88795704910348

In [15]:
mse_2(convlstm_predict_1, data_test)

2.599238867746326

In [16]:
n_tests = convlstm_predict_1.shape[0]
n_tests

92

In [17]:
mse_convlstm = []

In [18]:
for i in range(n_tests):
    mse_convlstm.append(mse(data_test[i], convlstm_predict_1[i]))

In [19]:
if not os.path.exists('result'):
    os.makedirs('result')

In [21]:
def save_data_pickle(data, path):
    with open(path, 'wb') as f:
        pickle.dump(data, f)

def load_data_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

In [23]:
save_data_pickle(mse_convlstm, 'result/convlstm.dat')

In [24]:
save_data_pickle(convlstm_predict_1, 'result/convlstm_predict_1.dat')